In [1]:
from roboflow import Roboflow
import json

rf = Roboflow(api_key="6Y4RTIEZtXjfjclMhIx1")
project = rf.workspace("ki-bu8lz").project("ki-dybmq")
version = project.version(7)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
import os
import shutil

# Das aktuelle Arbeitsverzeichnis ermitteln
dir_path = os.getcwd()

# Pfade für Trainings- und Validierungsordner definieren
input_path_train = os.path.join(dir_path, f"KI-{version.version}", "train")
input_path_valid = os.path.join(dir_path, f"KI-{version.version}", "valid")  # Rechtschreibung korrigiert: 'valid' statt 'vaild'
output_path_train = os.path.join(dir_path, "dataset_project", "images", "train") 
output_path_valid = os.path.join(dir_path, "dataset_project", "images", "valid")  # Rechtschreibung korrigiert: 'valid' statt 'vaild'

# Funktion zum Kopieren der Bilder und JSON-Dateien
def copy_files(source_folder, target_folder):
    # Erstellen des Zielordners, falls er nicht existiert
    os.makedirs(target_folder, exist_ok=True)
    
    # Liste der Dateien im Quellordner abrufen
    file_names = os.listdir(source_folder)
    
    # Kopieren der Dateien (Bilder und JSON-Dateien)
    for filename in file_names:
        source_path = os.path.join(source_folder, filename)
        target_path = os.path.join(target_folder, filename)
        
        # Wenn es sich um eine Bilddatei oder eine JSON-Datei handelt, kopieren
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.json')):
            try:
                shutil.copy(source_path, target_path)
                print(f"Datei '{filename}' wurde nach '{target_path}' kopiert.")
            except FileNotFoundError:
                print(f"Datei '{filename}' wurde nicht gefunden: {source_path}")
            except Exception as e:
                print(f"Fehler beim Kopieren von '{filename}': {e}")

# Trainingsbilder und Annotationsdateien kopieren
copy_files(input_path_train, output_path_train)

# Validierungsbilder und Annotationsdateien kopieren
copy_files(input_path_valid, output_path_valid)


In [ ]:
import os
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import json
from tqdm import tqdm

# Basisordner
output_path = os.path.join(dir_path, "dataset_project")
input_path = os.path.join("KI-" + version.version)

# Eingabeordner
train_images_folder = os.path.join(output_path, "images", "train")
annotations_file = os.path.join(train_images_folder, "_annotations.coco.json")

# Albumentations-Transformationspipeline
transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=30, p=0.7),
        A.RandomBrightnessContrast(p=0.4),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.5, p=1.0),  # Kontrast und Helligkeit verändern
        ToTensorV2()
    ],
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids'])  # Bounding Box Transformation
)

# Anzahl der augmentierten Bilder pro Originalbild
augmentation_count = 3

# Lade die Annotationen
with open(annotations_file, 'r') as f:
    annotations_data = json.load(f)

# Erstelle ein Mapping der Bild-ID auf die Annotationen
image_annotations_map = {}
for ann in annotations_data['annotations']:
    if ann['image_id'] not in image_annotations_map:
        image_annotations_map[ann['image_id']] = []
    image_annotations_map[ann['image_id']].append(ann)

# Erstelle ein Mapping der Bild-ID auf den Dateinamen
image_id_to_filename = {image['id']: image['file_name'] for image in annotations_data['images']}

# Aktualisiere die zentrale JSON-Datei
all_annotations = annotations_data

# Augmentierung der Bilder
for image_file in tqdm(os.listdir(train_images_folder)):
    if not image_file.endswith((".jpg", ".png", ".jpeg")):  # Nur Bilddateien bearbeiten
        continue

    image_path = os.path.join(train_images_folder, image_file)

    # Hole die Bild-ID aus der Zuordnung
    image_id = None
    for img_id, filename in image_id_to_filename.items():
        if filename == image_file:
            image_id = img_id
            break
    if image_id is None:
        print(f"Keine Bild-ID gefunden für Bild: {image_file}")
        continue

    # Bild laden
    image = cv2.imread(image_path)
    if image is None:
        print(f"Bild konnte nicht geladen werden: {image_file}")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Konvertierung von BGR zu RGB

    # Hole die zugehörigen Annotationen
    annotations = image_annotations_map.get(image_id, [])

    # Bounding Boxen und Labels extrahieren
    bboxes = []
    category_ids = []
    for ann in annotations:
        x, y, w, h = ann['bbox']
        bboxes.append([x, y, x + w, y + h])  # Umwandlung in [xmin, ymin, xmax, ymax]
        category_ids.append(ann['category_id'] - 1)  # YOLO Labels beginnen bei 0

    # Augmentierung durchführen
    for i in range(augmentation_count):
        augmented = transform(image=image, bboxes=bboxes, category_ids=category_ids)
        aug_image = augmented["image"]
        aug_bboxes = augmented["bboxes"]

        # Umwandlung des Tensors in ein NumPy-Array
        aug_image = aug_image.permute(1, 2, 0).numpy()  # Ändert (C, H, W) zu (H, W, C)
        aug_image = np.clip(aug_image, 0, 255).astype(np.uint8)  # Sicherstellen, dass Werte zwischen 0 und 255 sind

        # Speichern des augmentierten Bildes
        aug_image_filename = f"aug_{i}_{image_file}"
        aug_image_path = os.path.join(train_images_folder, aug_image_filename)
        aug_image = cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR)  # Zurück zu BGR für OpenCV
        cv2.imwrite(aug_image_path, aug_image)

        # Speichern der Annotationen für das augmentierte Bild
        aug_annotations = []
        for bbox, category_id in zip(aug_bboxes, category_ids):
            xmin, ymin, xmax, ymax = bbox
            w = xmax - xmin
            h = ymax - ymin
            aug_annotations.append({
                'category_id': category_id + 1,  # Wieder zurück zu originalem Format (1-basierend)
                'bbox': [xmin, ymin, w, h],
                'image_id': len(all_annotations['images']) + 1,
                'id': len(all_annotations['annotations']) + 1
            })

        # Füge die augmentierten Annotationen hinzu
        all_annotations['annotations'].extend(aug_annotations)

        # Füge die augmentierten Bildinformationen hinzu
        all_annotations['images'].append({
            'id': len(all_annotations['images']) + 1,
            'file_name': aug_image_filename,
            'height': image.shape[0],
            'width': image.shape[1]
        })

# Überschreibe die zentrale JSON-Datei im selben Ordner
with open(annotations_file, 'w') as json_file:
    json.dump(all_annotations, json_file)

print("Augmentierung abgeschlossen! Alle Dateien und die aktualisierte Annotation befinden sich im selben Ordner.")


In [ ]:
import os
import json

output_path = os.path.join(dir_path, "dataset_project")

# Angenommene Pfade für die JSON-Daten
file_path_train = os.path.join(output_path, "images", "train", "_annotations.coco.json")
file_path_valid = os.path.join(output_path, "images", "valid", "_annotations.coco.json")

# Ausgabeordner für die Labels
output_path = os.path.join(output_path)
train_images_folder = os.path.join(output_path, "images", "train")
valid_images_folder = os.path.join(output_path, "images", "valid")

# Annahme: Die Daten werden aus einer bereits geladenen COCO JSON-Datei extrahiert
with open(file_path_train, 'r') as f:
    data_train = json.load(f)

with open(file_path_valid, 'r') as f:
    data_valid = json.load(f)

# Funktion zum Abrufen der Annotationen für ein Bild
def get_img_ann(image_id, data):
    img_ann = []
    isFound = False
    for ann in data['annotations']:  # Verwendung des entsprechenden Datensatzes
        if ann['image_id'] == image_id:
            img_ann.append(ann)
            isFound = True
    return img_ann if isFound else None

# Funktion zum Abrufen der Bildinformationen anhand des Dateinamens
def get_img(filename, data):
    for img in data['images']:  # Verwendung des entsprechenden Datensatzes
        if img['file_name'] == filename:
            return img
    return None

# Definieren des Ausgabeordners für die Labels
labels_dir_train = os.path.join(output_path, "labels", "train")
labels_dir_valid = os.path.join(output_path, "labels", "valid")

if not os.path.exists(labels_dir_train):
    os.makedirs(labels_dir_train)  # Sicherstellen, dass der Ordner existiert

if not os.path.exists(labels_dir_valid):
    os.makedirs(labels_dir_valid)  # Sicherstellen, dass der Ordner existiert

# Bilddateinamen aus den Trainings- und Validierungsordnern abrufen
file_names_train = os.listdir(train_images_folder)  # Namen der Trainingsbilder
file_names_valid = os.listdir(valid_images_folder)  # Namen der Validierungsbilder

# Zähler für die Bilder
count = 0

# Funktion, um Label-Dateien zu erstellen
def process_images(file_names, images_folder, labels_dir, data):
    global count
    for filename in file_names:
        print(f"Verarbeite Bild: {filename}")

        # Bildinformationen abrufen
        img = get_img(filename, data)
        
        if img:
            img_id = img['id']
            img_w = img['width']
            img_h = img['height']

            # Annotationen für dieses Bild abrufen
            img_ann = get_img_ann(img_id, data)

            if img_ann:
                # Label-Datei im YOLO-Format erstellen
                label_file_path = os.path.join(labels_dir, filename.replace('.jpg', '.txt').replace('.png', '.txt'))
                print(f"Erstelle Label-Datei: {label_file_path}")
                with open(label_file_path, "a") as file_object:
                    for ann in img_ann:
                        current_category = ann['category_id'] - 1  # YOLO-Label beginnen bei 0
                        current_bbox = ann['bbox']
                        x = current_bbox[0]
                        y = current_bbox[1]
                        w = current_bbox[2]
                        h = current_bbox[3]

                        # Berechnung der Mittelpunkte
                        x_centre = (x + (x + w)) / 2
                        y_centre = (y + (y + h)) / 2

                        # Normalisierung der Werte
                        x_centre = x_centre / img_w
                        y_centre = y_centre / img_h
                        w = w / img_w
                        h = h / img_h

                        # Formatieren auf 6 Dezimalstellen
                        x_centre = format(x_centre, '.6f')
                        y_centre = format(y_centre, '.6f')
                        w = format(w, '.6f')
                        h = format(h, '.6f')

                        # Schreiben der Annotation ins Label-File
                        file_object.write(f"{current_category} {x_centre} {y_centre} {w} {h}\n")

                count += 1  # Zähler für Bildnummer erhöhen
            else:
                print(f"Keine Annotationen für das Bild: {filename}")
        else:
            print(f"Bild nicht gefunden: {filename}")

# Verarbeite sowohl Trainings- als auch Validierungsbilder
print("Verarbeite Trainingsbilder...")
process_images(file_names_train, train_images_folder, labels_dir_train, data_train)

print("Verarbeite Validierungsbilder...")
process_images(file_names_valid, valid_images_folder, labels_dir_valid, data_valid)

print("Alle Annotationen wurden verarbeitet.")


In [ ]:
import json
import os
import yaml

# Verzeichnis für die Ausgabedaten
output_path = os.path.join(dir_path, "dataset_project")
dir_path = os.getcwd()

# Eingabepfade für COCO JSON-Dateien
file_path_train = os.path.join(output_path, "images", "train", "_annotations.coco.json")
file_path_valid = os.path.join(output_path, "images", "valid", "_annotations.coco.json")

# Verzeichnisse für die Trainings- und Validierungsbilder und Labels
yaml_img_train = os.path.join(output_path, "images", "train")
yaml_img_val = os.path.join(output_path, "images", "valid")
yaml_label_train = os.path.join(output_path, "labels", "train")
yaml_label_valid = os.path.join(output_path, "labels", "valid")

# JSON-Dateien laden
with open(file_path_train, 'r') as f:
    data_train = json.load(f)

with open(file_path_valid, 'r') as f:
    data_valid = json.load(f)

# Kategorien extrahieren
category_names = [category['name'] for category in data_train['categories']]

# Erstellen der YAML-Datenstruktur
yaml_data = {
    "train": './images/train',         # Pfad zu den Trainingsbildern
    "val": './images/valid',           # Pfad zu den Validierungsbildern
    "train_labels": './labels/train',  # Pfad zu den Trainingslabels
    "val_labels": './labels/valid',    # Pfad zu den Validierungslabels
    "nc": len(category_names),        # Anzahl der Kategorien
    "names": category_names,          # Liste der Kategorienamen
    "batch": 8                         # Batch-Größe hinzufügen (z.B. 8)
}

# Pfad zur YAML-Datei
yaml_file_path = os.path.join(output_path, 'annotations.yaml')

# YAML-Datei speichern
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False, allow_unicode=True)

print(f"YAML-Datei wurde erfolgreich gespeichert: {yaml_file_path}")


In [1]:
import torch
#print("CUDA verfügbar:", torch.cuda.is_available())
#print("cuDNN Version:", torch.backends.cudnn.version())
#print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Keine GPU gefunden")
torch.cuda.device_count()




: 

In [ ]:
from ultralytics import YOLO
import requests
import os
import torch
print("CUDA verfügbar:", torch.cuda.is_available())

output_path = os.path.join(dir_path, "dataset_project")
data_path = os.path.join(output_path, "annotations.yaml")

# YOLOv8-Modell laden
model = YOLO("yolov8n.pt")  # Modell: n (Nano), s (Small), m (Medium), etc.

# Training starten
model.train(
    data=(data_path),  # Pfad zur YAML-Datei
    epochs=1,            # Anzahl der Epochen
    imgsz=640,             # Bildgröße (anpassen bei Bedarf)
)

print("Training abgeschlossen!")


In [ ]:
# URL des Bildes
image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQXWZd5b-rXfKMONr01EAgXsTZ0Sm_BMRAXvQ&s"

# Lokale Datei speichern
local_image_path = "downloaded_image.jpg"
response = requests.get(image_url)

if response.status_code == 200:
    # Bild lokal speichern
    with open(local_image_path, "wb") as file:
        file.write(response.content)
    print(f"Bild erfolgreich heruntergeladen: {local_image_path}")
else:
    print(f"Fehler beim Herunterladen des Bildes: {response.status_code}")

# Trainiertes YOLO-Modell laden
model = YOLO(r"C:\Users\thore\Documents\GitHub\ki\runs\detect\train12\weights\best.pt")

# Inferenz auf dem heruntergeladenen Bild
results = model(local_image_path)

# Ergebnisse anzeigen
# results ist eine Liste von Resultaten, daher durchlaufen wir sie.
for result in results:
    result.show()  # Zeigt das Bild mit erkannten Objekten

# Ergebnisse speichern
for result in results:
    result.save("predictions/")  # Speichert die analysierten Bilder


In [ ]:
import cv2
import torch
from pathlib import Path

# Load the YOLOv8 model (choose the pre-trained model you want)
model = torch.hub.load('ultralytics/yolov8', 'yolov8n')  # Replace 'yolov8n' with your desired model, such as 'yolov8s', 'yolov8m', or 'yolov8l'

# Open the video file
video_path = 'input_video.mp4'  # Path to your input video
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the output video file name
output_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection on the current frame
    results = model(frame)  # Perform detection on the current frame

    # Render the results on the frame
    frame = results.render()[0]  # results.render() modifies the frame

    # Write the processed frame to the output video
    out.write(frame)

    # Display the frame (optional)
    cv2.imshow('Object Detection', frame)

    # Press 'q' to quit early
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video objects
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Object detection complete. Output saved to {output_path}")
